In [1]:
import pandas as pd
import numpy as np
import scipy
import os
import re
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import pickle



import gensim
from gensim import corpora, models, matutils
from gensim.models import Word2Vec
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction import _stop_words
from nltk.stem import WordNetLemmatizer


from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\harsh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\harsh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\harsh\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
df=pd.read_csv('stock_data_nlp.csv',encoding= 'unicode_escape')
df.head(2)

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2000-01-03,0,A 'hindrance to operations': extracts from the...,Scorecard,Hughes' instant hit buoys Blues,Jack gets his skates on at ice-cold Alex,Chaos as Maracana builds up for United,Depleted Leicester prevail as Elliott spoils E...,Hungry Spurs sense rich pickings,Gunners so wide of an easy target,...,Flintoff injury piles on woe for England,Hunters threaten Jospin with new battle of the...,Kohl's successor drawn into scandal,The difference between men and women,"Sara Denver, nurse turned solicitor",Diana's landmine crusade put Tories in a panic,Yeltsin's resignation caught opposition flat-f...,Russian roulette,Sold out,Recovering a title
1,2000-01-04,0,Scorecard,The best lake scene,Leader: German sleaze inquiry,"Cheerio, boyo",The main recommendations,Has Cubie killed fees?,Has Cubie killed fees?,Has Cubie killed fees?,...,On the critical list,The timing of their lives,Dear doctor,Irish court halts IRA man's extradition to Nor...,Burundi peace initiative fades after rebels re...,PE points the way forward to the ECB,Campaigners keep up pressure on Nazi war crime...,Jane Ratcliffe,Yet more things you wouldn't know without the ...,Millennium bug fails to bite


In [3]:
df.iloc[0,4]

"Hughes' instant hit buoys Blues"

In [4]:
df.shape

(4262, 27)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4262 entries, 0 to 4261
Data columns (total 27 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    4262 non-null   object
 1   Label   4214 non-null   object
 2   Top1    4210 non-null   object
 3   Top2    4202 non-null   object
 4   Top3    4192 non-null   object
 5   Top4    4169 non-null   object
 6   Top5    4158 non-null   object
 7   Top6    4151 non-null   object
 8   Top7    4144 non-null   object
 9   Top8    4135 non-null   object
 10  Top9    4127 non-null   object
 11  Top10   4118 non-null   object
 12  Top11   4106 non-null   object
 13  Top12   4100 non-null   object
 14  Top13   4091 non-null   object
 15  Top14   4083 non-null   object
 16  Top15   4075 non-null   object
 17  Top16   4071 non-null   object
 18  Top17   4064 non-null   object
 19  Top18   4054 non-null   object
 20  Top19   4049 non-null   object
 21  Top20   4040 non-null   object
 22  Top21   4030 non-null   

In [6]:
df.Label.value_counts()

1                                                                                                                                                                                                                                                                                                     2166
0                                                                                                                                                                                                                                                                                                     1935
b'FBI arrests 10 Russian Spies in U.S.'                                                                                                                                                                                                                                                                  1
b'PHILIPPINES Key witness in the Maguindanao massacre case killed'                                     

In [7]:
#remove the date column
#the stock close trend is directly affected by the news from the same day
df=df.drop(['Date'],axis=1)

In [8]:
df.head()

,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,Top9,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,0,A 'hindrance to operations': extracts from the...,Scorecard,Hughes' instant hit buoys Blues,Jack gets his skates on at ice-cold Alex,Chaos as Maracana builds up for United,Depleted Leicester prevail as Elliott spoils E...,Hungry Spurs sense rich pickings,Gunners so wide of an easy target,Derby raise a glass to Strupar's debut double,...,Flintoff injury piles on woe for England,Hunters threaten Jospin with new battle of the...,Kohl's successor drawn into scandal,The difference between men and women,"Sara Denver, nurse turned solicitor",Diana's landmine crusade put Tories in a panic,Yeltsin's resignation caught opposition flat-f...,Russian roulette,Sold out,Recovering a title
1,0,Scorecard,The best lake scene,Leader: German sleaze inquiry,"Cheerio, boyo",The main recommendations,Has Cubie killed fees?,Has Cubie killed fees?,Has Cubie killed fees?,Hopkins 'furious' at Foster's lack of Hannibal...,...,On the critical list,The timing of their lives,Dear doctor,Irish court halts IRA man's extradition to Nor...,Burundi peace initiative fades after rebels re...,PE points the way forward to the ECB,Campaigners keep up pressure on Nazi war crime...,Jane Ratcliffe,Yet more things you wouldn't know without the ...,Millennium bug fails to bite
2,0,Coventry caught on counter by Flo,United's rivals on the road to Rio,Thatcher issues defence before trial by video,Police help Smith lay down the law at Everton,Tale of Trautmann bears two more retellings,England on the rack,Pakistan retaliate with call for video of Walsh,Cullinan continues his Cape monopoly,McGrath puts India out of their misery,...,South Melbourne (Australia),Necaxa (Mexico),Real Madrid (Spain),Raja Casablanca (Morocco),Corinthians (Brazil),Tony's pet project,Al Nassr (Saudi Arabia),Ideal Holmes show,Pinochet leaves hospital after tests,Useful links
3,1,Pilgrim knows how to progress,Thatcher facing ban,McIlroy calls for Irish fighting spirit,Leicester bin stadium blueprint,United braced for Mexican wave,"Auntie back in fashion, even if the dress look...",Shoaib appeal goes to the top,Hussain hurt by 'shambles' but lays blame on e...,England's decade of disasters,...,Putin admits Yeltsin quit to give him a head s...,BBC worst hit as digital TV begins to bite,How much can you pay for...,Christmas glitches,"Upending a table, Chopping a line and Scoring ...","Scientific evidence 'unreliable', defence claims",Fusco wins judicial review in extradition case,Rebels thwart Russian advance,Blair orders shake-up of failing NHS,Lessons of law's hard heart
4,1,Hitches and Horlocks,Beckham off but United survive,Breast cancer screening,Alan Parker,Guardian readers: are you all whingers?,Hollywood Beyond,Ashes and diamonds,Whingers - a formidable minority,Alan Parker - part two,...,Most everywhere: UDIs,Most wanted: Chloe lunettes,Return of the cane 'completely off the agenda',From Sleepy Hollow to Greeneland,Blunkett outlines vision for over 11s,"Embattled Dobson attacks 'play now, pay later'...",Doom and the Dome,What is the north-south divide?,Aitken released from jail,Gone aloft


In [9]:
# combine all 25 news of the same day into a long string for doc2vec and vectorize
combine_news=[]

for index,row in df.iterrows():
    combine_news.append(' '.join(str(x) for x in row[1:]))

df['news']=pd.DataFrame(combine_news)
    

In [10]:
df.loc[1,'news']

"Scorecard The best lake scene Leader: German sleaze inquiry Cheerio, boyo The main recommendations Has Cubie killed fees? Has Cubie killed fees? Has Cubie killed fees? Hopkins 'furious' at Foster's lack of Hannibal appetite Has Cubie killed fees? A tale of two tails I say what I like and I like what I say Elbows, Eyes and Nipples Task force to assess risk of asteroid collision How I found myself at last On the critical list The timing of their lives Dear doctor Irish court halts IRA man's extradition to Northern Ireland Burundi peace initiative fades after rebels reject Mandela as mediator PE points the way forward to the ECB Campaigners keep up pressure on Nazi war crimes suspect Jane Ratcliffe Yet more things you wouldn't know without the movies Millennium bug fails to bite"

In [11]:
# Find all the 2/3 character abbrevations from the string corpus. Convert them to full form. 
# Then we can remove the punctuations without worrying losing the meaning of the abbrevation words.

# combine all the news into a very long string
long_news_str = '   '.join(df.news)

# find all the abbrevations of 2 and 3 characters
def find_abbr(text):
    abbr = []
    for i in re.finditer(r"([A-Za-z]+| )([A-Za-z]\.){2,}", text):
        abbr.append(i.group())
    df_abbr = pd.Series(abbr)
    return df_abbr.unique()

find_abbr(long_news_str)

array([' A.I.', ' M.A.N.D.Y.', ' U.N.', ' U.S.', ' U.K.', ' S.A.',
       ' U.S.C.', ' D.C.', ' N.J.', ' i.e.', ' P.I.', ' A.N.C.', ' a.m.',
       ' A.K.A.', ' P.R.', ' R.I.', ' E.U.', ' H.I.V.', ' I.H.T.',
       ' B.C.', ' J.P.', ' N.S.', 'crimese.g.', ' C.I.A.', ' p.m.',
       'Ph.D.', ' N.Y.', ' U.A.E.', 'sq.m.', ' I.M.F.', ' y.o.', ' i.a.',
       ' I.D.', ' M.A.', ' H.W.', ' O.K.', ' N.K.', ' B.S.', ' A.T.M.',
       ' W.H.O.', ' N.S.A.', ' P.M.', ' F.B.I.', ' P.E.I.', ' a.k.a.',
       ' S.E.', ' A.D.', ' T.B.', ' J.K.', ' L.G.B.T.'], dtype=object)

In [12]:
# get ready stop words for the tokenization function below
mywords = ['breaking','whilst', 'say', 'says', 'today','yesterday', 'news', 'tomorrow','iii', 'ii', 'like', 'ha',]
final_stop = stopwords.words('english') + mywords

In [13]:
def text_to_wordlist(text, remove_stop_words=True, lemma_words=True):
    
    ''' Clean each document into a list of words:
    1. convert abbrevations to full words
    2. tokenize the text
    3. remove non-alphabetic characters and one-letter words, including numbers and punctuations
    4. remove stop words
    '''
    # clean the text, convert only the abbrs that are meaningful
    text = re.sub(r" A.T.M. ", " Automated Teller Machine ", text)
    text = re.sub(r" C.I.A. ", " Central Intelligence Agency ", text)
    text = re.sub(r" D.C. ", " District of columbia ", text)
    text = re.sub(r" E.U. ", " Europian Union ", text)
    text = re.sub(r" F.B.I. ", " Federal Bureau of Investigation ", text)
    text = re.sub(r" H.I.V. ", " Human immunodeficiency virus ", text)
    text = re.sub(r" I.H.T. ", " inheritance tax ", text)
    text = re.sub(r" I.M.F. ", " International Monetary Fund ", text)
    text = re.sub(r" I.D. ", " identification ", text)
    text = re.sub(r" L.G.B.T. ", " minority ", text)
    text = re.sub(r" M.A. ", " Massachusetts ", text)
    text = re.sub(r" N.J. ", " new jersey ", text)
    text = re.sub(r" N.K. ", " north korea ", text)
    text = re.sub(r" N.S.A. ", " National Security Agency ", text)
    text = re.sub(r" N.Y. ", " new york ", text)
    text = re.sub(r" P.E.I. ", " Prince Edward Island ", text)
    text = re.sub(r" P.M. ", " prime minister ", text)
    text = re.sub(r" P.R.C ", " china ", text)
    text = re.sub(r" S.A. ", " south africa ", text)
    text = re.sub(r" R.I. ", " Rhode Island ", text)
    text = re.sub(r" U.A.E. ", " United Arab Emirates ", text)
    text = re.sub(r" U.K. ", " england ", text)
    text = re.sub(r" U.N. ", " new jersey ", text)
    text = re.sub(r" U.S. ", " america ", text)
    text = re.sub(r" U.S.C. ", " university of south california ", text)
    text = re.sub(r" W.H.O ", " world health organization ", text)
    text = re.sub(r" a.m. ", " morning ", text)
    text = re.sub(r" p.m. ", " afternoon ", text)
    text = re.sub(r" Ph.D. ", " doctor of philosophy ", text)
    text = re.sub(r" sq.m. ", " square meter ", text)
    
    # Tokenize the string into word tokens
    tokens = word_tokenize(text)
    
    # further clean the tokens: split toekns like "b'Russia" which still have punctuations in the token
    ls = []
    for word in tokens:
        if "'" in word:
            ls = ls + word.split("'")
    tokens = tokens + ls
    
    # Optionally, shorten words to their stems
    if lemma_words:
        tokens = [WordNetLemmatizer().lemmatize(word) for word in tokens]
    
    #Remove one letter tokens & non-alphabetic tokens, such as punctuation, then lower the tokens
    tokens = [word.lower() for word in tokens if (word.isalpha() and len(word)>1)]

    # remove stop words  
    if remove_stop_words:
        tokens = [word for word in tokens if word not in final_stop]
               
    return tokens

In [14]:
# convert each document to list of words
df.news = df.news.apply(lambda x: text_to_wordlist(x))

In [15]:
df.news[1]

['scorecard',
 'best',
 'lake',
 'scene',
 'leader',
 'german',
 'sleaze',
 'inquiry',
 'cheerio',
 'boyo',
 'main',
 'recommendation',
 'cubie',
 'killed',
 'fee',
 'cubie',
 'killed',
 'fee',
 'cubie',
 'killed',
 'fee',
 'hopkins',
 'foster',
 'lack',
 'hannibal',
 'appetite',
 'cubie',
 'killed',
 'fee',
 'tale',
 'two',
 'tail',
 'elbows',
 'eyes',
 'nipples',
 'task',
 'force',
 'ass',
 'risk',
 'asteroid',
 'collision',
 'found',
 'last',
 'critical',
 'list',
 'timing',
 'life',
 'dear',
 'doctor',
 'irish',
 'court',
 'halt',
 'ira',
 'man',
 'extradition',
 'northern',
 'ireland',
 'burundi',
 'peace',
 'initiative',
 'fade',
 'rebel',
 'reject',
 'mandela',
 'mediator',
 'pe',
 'point',
 'way',
 'forward',
 'ecb',
 'campaigners',
 'keep',
 'pressure',
 'nazi',
 'war',
 'crime',
 'suspect',
 'jane',
 'ratcliffe',
 'yet',
 'thing',
 'would',
 'know',
 'without',
 'movie',
 'millennium',
 'bug',
 'fails',
 'bite',
 'furious']

In [16]:
#save it in a binary format for future use
with open('news_wordlists.pkl', 'wb') as f:
    pickle.dump(df, f)


## Word2Vec

In [17]:
import gensim.downloader as api

In [18]:
corpus=api.load('text8')

In [19]:
from gensim.models.word2vec import Word2Vec
model = Word2Vec(corpus)

In [20]:
economy=model.wv.get_vector('south')
print(economy)

[-2.509381   -0.16613127 -0.8992871  -2.9068344   1.4750314   0.80374986
  0.7952001   0.7238951   1.6035595  -0.02354915  3.0688202   2.5213227
  1.0252578  -0.05220444 -1.1082993  -1.2946367   0.66607404 -0.19928414
 -0.97570974  1.626208    1.2356777   0.7779477  -1.0477018   2.0057783
 -2.4510858   3.0472026  -1.0234712   1.9142127   0.14806604  2.4640114
 -0.23066552 -3.6230218   2.607124    1.1380025  -2.2711272  -2.235657
 -0.7185839   0.6535337   0.67939144  1.8852011   0.72671854 -0.70859224
  3.2457647  -1.0079061   1.5675293  -1.6929959   0.32602417 -0.97245795
 -0.35630977 -1.1249006   2.1012487   2.1822665   1.8151661  -0.6936743
 -0.6782292   2.1006563   1.076847    1.4930735   0.6676121   0.17348084
 -0.8526461  -0.5471324   0.49434534  1.169552   -0.16901694  3.1406465
  1.688444    1.5923692   0.3708412  -1.9049963  -0.8629914   0.90316373
  0.7333295  -3.636362    0.48794597 -1.7392259  -0.21886991  2.5903301
 -0.15998559  1.3041346  -0.8568234  -1.6246411   0.2222131

In [21]:






def doc2vec(model,wordlist):
    vector_list=[model.wv.get_vector(word) for word in wordlist if word in model.wv]
    doc_vector=np.mean(vector_list,axis=0)
    return doc_vector


In [22]:
# convert each document (list of words) to a document vecter, then save into a list of doc_vec
x_doc = [doc2vec(model, doc) for doc in df.news]  
X_doc= pd.Series(x_doc, name = 'doc_vec') # list to df

In [23]:
X_doc.shape

(4262,)

In [24]:
# a new dataframe containing only label, news, doc_vec
newdf = pd.concat([df.Label,df.news,X_doc], axis = 1)

# convert each row in the news column from a list of tokens to a string
tokenstrlist = []
for tokenlist in  df.news:
    tokenstr = ' '.join(tokenlist)
    tokenstrlist.append(tokenstr)

# add the string format new as clolumn 'news_str' to the data frame
newdf['news_str'] = pd.Series(tokenstrlist)

# the final data frame is ready for modeling
newdf.head()



,Label,news,doc_vec,news_str
0,0,"[operation, extract, leaked, report, scorecard...","[-0.13853689, -0.01842522, -0.011283746, 0.025...",operation extract leaked report scorecard hugh...
1,0,"[scorecard, best, lake, scene, leader, german,...","[-0.13274541, -0.32202816, 0.17546004, 0.15300...",scorecard best lake scene leader german sleaze...
2,0,"[coventry, caught, counter, flo, united, rival...","[-0.41247106, 0.4019597, 0.16694483, 0.0331079...",coventry caught counter flo united rival road ...
3,1,"[pilgrim, know, progress, thatcher, facing, ba...","[-0.232915, -0.15494952, 0.13468608, -0.152680...",pilgrim know progress thatcher facing ban mcil...
4,1,"[hitches, horlocks, beckham, united, survive, ...","[-0.33469847, -0.034216043, 0.117651, -0.06944...",hitches horlocks beckham united survive breast...


In [25]:
newdf = newdf[newdf['Label'].isin(['0', '1'])]

In [26]:
newdf['Label'].dtype

dtype('O')

In [27]:

newdf['Label'] = newdf['Label'].astype(int)


In [28]:
with open('label_news_docvec_newsstr.pkl', 'wb') as f:
    pickle.dump(newdf, f)

### Finding the best baseline Model
##### every dataset is different
We dont know which model works best for our data until we try. 

In [29]:
from sklearn.metrics import classification_report


In [30]:
with open('label_news_docvec_newsstr.pkl', 'rb') as r:
    df = pickle.load(r)
df.head()

,Label,news,doc_vec,news_str
0,0,"[operation, extract, leaked, report, scorecard...","[-0.13853689, -0.01842522, -0.011283746, 0.025...",operation extract leaked report scorecard hugh...
1,0,"[scorecard, best, lake, scene, leader, german,...","[-0.13274541, -0.32202816, 0.17546004, 0.15300...",scorecard best lake scene leader german sleaze...
2,0,"[coventry, caught, counter, flo, united, rival...","[-0.41247106, 0.4019597, 0.16694483, 0.0331079...",coventry caught counter flo united rival road ...
3,1,"[pilgrim, know, progress, thatcher, facing, ba...","[-0.232915, -0.15494952, 0.13468608, -0.152680...",pilgrim know progress thatcher facing ban mcil...
4,1,"[hitches, horlocks, beckham, united, survive, ...","[-0.33469847, -0.034216043, 0.117651, -0.06944...",hitches horlocks beckham united survive breast...


In [31]:
df.iloc[0,2][:20]

array([-0.13853689, -0.01842522, -0.01128375,  0.02521024, -0.12980145,
       -0.4068818 ,  0.13044144,  0.25541753,  0.37049475,  0.09962238,
       -0.23451243, -0.27692914,  0.24458377,  0.31107265, -0.11690012,
       -0.5110352 ,  0.0377483 , -0.4658377 ,  0.45661828, -0.18214466],
      dtype=float32)

In [32]:
df.iloc[0,2][:20]

array([-0.13853689, -0.01842522, -0.01128375,  0.02521024, -0.12980145,
       -0.4068818 ,  0.13044144,  0.25541753,  0.37049475,  0.09962238,
       -0.23451243, -0.27692914,  0.24458377,  0.31107265, -0.11690012,
       -0.5110352 ,  0.0377483 , -0.4658377 ,  0.45661828, -0.18214466],
      dtype=float32)

In [33]:
df.iloc[0,3]

'operation extract leaked report scorecard hughes instant hit buoy blues jack get skate alex chaos maracana build united depleted leicester prevail elliott spoil everton party hungry spurs sense rich picking gunners wide easy target derby raise glass strupar debut double southgate strike leeds pay penalty hammers hand robson youthful lesson saints party wear wolf turned lamb stump mike catch testy gough taunt langer escape hit flintoff injury pile woe england hunters threaten jospin new battle somme kohl successor drawn scandal difference men woman sara denver nurse turned solicitor diana landmine crusade put tories panic yeltsin resignation caught opposition russian roulette sold recovering title hindrance'

In [34]:
df_train=df.iloc[:3623,:] #85% for training and validation, 15% for testing

# Models

In [35]:
from sklearn import  svm, naive_bayes, neighbors, ensemble
from sklearn.linear_model import LogisticRegression
import xgboost as xgb

lr_model = LogisticRegression()
nb_model = naive_bayes.GaussianNB()
knn_model = neighbors.KNeighborsClassifier()
svc_model = svm.SVC(probability=True, gamma="scale")
rf_model = ensemble.RandomForestClassifier(n_estimators=100)
et_model = ensemble.ExtraTreesClassifier(n_estimators=100)
ada_model = ensemble.AdaBoostClassifier()
xgb_model = xgb.XGBClassifier(max_depth=50, n_estimators=80, learning_rate=0.1, colsample_bytree=.7, gamma=0, 
                              reg_alpha=4, objective='binary:logistic', eta=0.3, silent=1, subsample=0.8)

models = ["lr_model", "nb_model", "knn_model", "svc_model", "rf_model", "et_model", "ada_model", "xgb_model"]

In [36]:
def baseline_model_filter(modellist, X, y):
    ''' 1. split the train data further into train and validation (17%). 
        2. fit the train data into each model of the model list
        3. get the classification report based on the model performance on validation data
    '''
    X_train, X_valid, y_train, y_valid = train_test_split(X,y, test_size = 0.17, random_state = 100)
    for model_name in modellist:
        curr_model = eval(model_name)
        curr_model.fit(X_train, y_train) 
        print(f'{model_name} \n report:{classification_report(y_valid, curr_model.predict(X_valid))}')

## Using Different word embedding techniques to filter the baseline models

### Bag-Of-Words

In [37]:
print(df_train['news_str'].isnull().sum())

0


In [38]:
count_vect = CountVectorizer(analyzer='word')
X = count_vect.fit_transform(df_train.news_str).toarray()
y = df_train.Label

In [39]:
baseline_model_filter(models, X, y)

C:\Users\harsh\anaconda3\envs\mygpu\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


lr_model 
 report:              precision    recall  f1-score   support

           0       0.45      0.44      0.45       284
           1       0.53      0.54      0.54       332

    accuracy                           0.50       616
   macro avg       0.49      0.49      0.49       616
weighted avg       0.49      0.50      0.49       616

nb_model 
 report:              precision    recall  f1-score   support

           0       0.48      0.58      0.53       284
           1       0.57      0.47      0.51       332

    accuracy                           0.52       616
   macro avg       0.53      0.53      0.52       616
weighted avg       0.53      0.52      0.52       616

knn_model 
 report:              precision    recall  f1-score   support

           0       0.50      0.18      0.26       284
           1       0.55      0.84      0.66       332

    accuracy                           0.54       616
   macro avg       0.52      0.51      0.46       616
weighted avg       

## Word level TF-IDF

In [40]:
tfidf_vect = TfidfVectorizer(analyzer='word')
X = tfidf_vect.fit_transform(df_train.news_str).toarray()
y = df_train.Label

In [41]:
baseline_model_filter(models, X, y)

lr_model 
 report:              precision    recall  f1-score   support

           0       0.47      0.30      0.37       284
           1       0.54      0.71      0.62       332

    accuracy                           0.52       616
   macro avg       0.51      0.51      0.49       616
weighted avg       0.51      0.52      0.50       616

nb_model 
 report:              precision    recall  f1-score   support

           0       0.49      0.51      0.50       284
           1       0.56      0.54      0.55       332

    accuracy                           0.53       616
   macro avg       0.52      0.52      0.52       616
weighted avg       0.53      0.53      0.53       616

knn_model 
 report:              precision    recall  f1-score   support

           0       0.47      0.50      0.49       284
           1       0.55      0.52      0.54       332

    accuracy                           0.51       616
   macro avg       0.51      0.51      0.51       616
weighted avg       

## Character Level TF-IDF

In [42]:
tfidf_chars_vect = TfidfVectorizer(analyzer='char')

X = tfidf_chars_vect.fit_transform(df_train.news_str).toarray()
y = df_train.Label

baseline_model_filter(models, X, y)

lr_model 
 report:              precision    recall  f1-score   support

           0       0.62      0.05      0.10       284
           1       0.55      0.97      0.70       332

    accuracy                           0.55       616
   macro avg       0.59      0.51      0.40       616
weighted avg       0.58      0.55      0.42       616

nb_model 
 report:              precision    recall  f1-score   support

           0       0.46      0.99      0.63       284
           1       0.62      0.02      0.03       332

    accuracy                           0.46       616
   macro avg       0.54      0.50      0.33       616
weighted avg       0.55      0.46      0.31       616

knn_model 
 report:              precision    recall  f1-score   support

           0       0.45      0.47      0.46       284
           1       0.53      0.52      0.52       332

    accuracy                           0.49       616
   macro avg       0.49      0.49      0.49       616
weighted avg       

C:\Users\harsh\anaconda3\envs\mygpu\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\harsh\anaconda3\envs\mygpu\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\harsh\anaconda3\envs\mygpu\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


svc_model 
 report:              precision    recall  f1-score   support

           0       0.00      0.00      0.00       284
           1       0.54      1.00      0.70       332

    accuracy                           0.54       616
   macro avg       0.27      0.50      0.35       616
weighted avg       0.29      0.54      0.38       616

rf_model 
 report:              precision    recall  f1-score   support

           0       0.47      0.42      0.44       284
           1       0.54      0.59      0.57       332

    accuracy                           0.51       616
   macro avg       0.51      0.51      0.51       616
weighted avg       0.51      0.51      0.51       616

et_model 
 report:              precision    recall  f1-score   support

           0       0.47      0.43      0.45       284
           1       0.54      0.58      0.56       332

    accuracy                           0.51       616
   macro avg       0.51      0.51      0.51       616
weighted avg       

### Word2vec

In [43]:
X = np.array(list(df_train.doc_vec))
y = np.array(list(df.Label[:3623]))
baseline_model_filter(models, X, y)

lr_model 
 report:              precision    recall  f1-score   support

           0       0.48      0.32      0.38       284
           1       0.55      0.70      0.61       332

    accuracy                           0.52       616
   macro avg       0.51      0.51      0.50       616
weighted avg       0.51      0.52      0.51       616

nb_model 
 report:              precision    recall  f1-score   support

           0       0.46      0.42      0.44       284
           1       0.54      0.57      0.55       332

    accuracy                           0.50       616
   macro avg       0.50      0.50      0.50       616
weighted avg       0.50      0.50      0.50       616

knn_model 
 report:              precision    recall  f1-score   support

           0       0.47      0.39      0.43       284
           1       0.55      0.63      0.59       332

    accuracy                           0.52       616
   macro avg       0.51      0.51      0.51       616
weighted avg       

C:\Users\harsh\AppData\Local\Temp\ipykernel_25072\3845775326.py:2: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  y = np.array(list(df.Label[:3623]))


svc_model 
 report:              precision    recall  f1-score   support

           0       0.41      0.13      0.20       284
           1       0.53      0.84      0.65       332

    accuracy                           0.51       616
   macro avg       0.47      0.49      0.43       616
weighted avg       0.48      0.51      0.44       616

rf_model 
 report:              precision    recall  f1-score   support

           0       0.49      0.42      0.45       284
           1       0.56      0.62      0.59       332

    accuracy                           0.53       616
   macro avg       0.52      0.52      0.52       616
weighted avg       0.52      0.53      0.52       616

et_model 
 report:              precision    recall  f1-score   support

           0       0.44      0.37      0.40       284
           1       0.53      0.60      0.56       332

    accuracy                           0.49       616
   macro avg       0.48      0.48      0.48       616
weighted avg       

### see performance on the test data

In [44]:
df_test = df.iloc[3623:,:]
y_test = df_test.Label
X_test = df_test.news_str

In [45]:
tfidf_chars_vect = TfidfVectorizer(analyzer='word')
train_news = tfidf_chars_vect.fit(df_train.news_str)
X_train_trans = train_news.transform(df_train.news_str).toarray()
X_test_trans = train_news.transform(X_test).toarray()

y_train = df_train.Label

nb_model.fit(X_train_trans, y_train) 


print(f'{nb_model} \n report:{classification_report(y_test, nb_model.predict(X_test_trans))}')

GaussianNB() 
 report:              precision    recall  f1-score   support

           0       0.52      0.47      0.49       229
           1       0.55      0.60      0.58       249

    accuracy                           0.54       478
   macro avg       0.54      0.53      0.53       478
weighted avg       0.54      0.54      0.54       478



In [46]:
with open('label_news_docvec_newsstr.pkl', 'rb') as r:
    df = pickle.load(r)
df.head()

,Label,news,doc_vec,news_str
0,0,"[operation, extract, leaked, report, scorecard...","[-0.13853689, -0.01842522, -0.011283746, 0.025...",operation extract leaked report scorecard hugh...
1,0,"[scorecard, best, lake, scene, leader, german,...","[-0.13274541, -0.32202816, 0.17546004, 0.15300...",scorecard best lake scene leader german sleaze...
2,0,"[coventry, caught, counter, flo, united, rival...","[-0.41247106, 0.4019597, 0.16694483, 0.0331079...",coventry caught counter flo united rival road ...
3,1,"[pilgrim, know, progress, thatcher, facing, ba...","[-0.232915, -0.15494952, 0.13468608, -0.152680...",pilgrim know progress thatcher facing ban mcil...
4,1,"[hitches, horlocks, beckham, united, survive, ...","[-0.33469847, -0.034216043, 0.117651, -0.06944...",hitches horlocks beckham united survive breast...


## Topic Modeling to vectors ( Unsupervised Learning)

#### Data Allocation
1. For Topic Modelling: 15% data set aside for testing. use the 85% for topic modelling
2. Apply the topic model to the testing data to get the topic vectors
3. Create the final train,valid, test files for AWS

Topic Modeling Models
1. Use HDP(Hierarchical Dirichlet Process) to decide topic size
2. Use LDA (Latent Dirichlet Allocation to determine the topics

In [47]:
df_HDP_train = df.news.iloc[:3623]
df_LDA_train = df.news_str.iloc[:3623]
df_LDA_test = df.news_str.iloc[3623:]
print(len(df_LDA_train), len(df_LDA_test))

3623 478


## HDP corpus and dictionary (need to be bag of words format)

In [48]:
id2word_hdp = gensim.corpora.Dictionary(df_HDP_train)
id2word_hdp.filter_extremes(no_below=10, no_above=0.30)
id2word_hdp.compactify()
id2word_hdp.save('train_dict_hdp')
corpus_hdp = [id2word_hdp.doc2bow(doc) for doc in df_HDP_train]

## Use HDP model to decide the maxium topic numbers

In [49]:
from gensim.models import HdpModel
hdp = HdpModel(corpus_hdp, id2word_hdp)

In [50]:
len(hdp.print_topics())

20

In [51]:
hdp.print_topics(num_topics=20)

[(0,
  '0.004*israel + 0.004*china + 0.003*people + 0.003*country + 0.003*russia + 0.003*war + 0.003*america + 0.003*uk + 0.003*killed + 0.003*one'),
 (1,
  '0.007*nan + 0.004*israel + 0.003*war + 0.003*people + 0.003*china + 0.003*israeli + 0.003*country + 0.003*one + 0.003*leader + 0.002*iran'),
 (2,
  '0.003*rugby + 0.003*football + 0.003*letters + 0.003*review + 0.003*cup + 0.003*cricket + 0.002*england + 0.002*israel + 0.002*union + 0.002*leader'),
 (3,
  '0.005*football + 0.005*review + 0.003*england + 0.003*cricket + 0.003*cd + 0.003*united + 0.002*cup + 0.002*back + 0.002*league + 0.002*rugby'),
 (4,
  '0.006*nan + 0.004*review + 0.002*football + 0.002*united + 0.002*letters + 0.002*war + 0.002*england + 0.002*uk + 0.002*city + 0.002*face'),
 (5,
  '0.007*nan + 0.003*review + 0.002*letters + 0.002*england + 0.002*london + 0.002*leader + 0.002*face + 0.002*rugby + 0.002*diary + 0.002*racing'),
 (6,
  '0.004*nan + 0.001*back + 0.001*admits + 0.001*face + 0.001*attempting + 0.001*

## LDA Modeling for topic vectors

In [52]:
from nltk.corpus import stopwords
from sklearn.feature_extraction import _stop_words
# we need to have more stopwords for topic modeling than for word2vec.
# NLTK + SKlearn + self definded
sk_stop = list(_stop_words.ENGLISH_STOP_WORDS)
mywords = ['whilst', 'say', 'says', 'today','yesterday', 'news', 'tomorrow','iii', 'ii', 'like', 'ha','wa']
final_stop = stopwords.words('english') + mywords + sk_stop

In [53]:
tfv = TfidfVectorizer(stop_words = final_stop, ngram_range = (1, 2), max_df = 0.95)
doc_word = tfv.fit_transform(df_LDA_train).transpose()
corpus = matutils.Sparse2Corpus(doc_word)
id2word = dict((v, k) for k, v in tfv.vocabulary_.items())

In [57]:
from gensim.models import LdaModel

In [58]:
import logging
logging.basicConfig(filename='lda_model.log', format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

import warnings
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    lda = LdaModel(corpus=corpus, num_topics=20, id2word=id2word, passes=5, random_state = 200 )

In [59]:
lda.print_topics()

[(0,
  '0.000*"world" + 0.000*"iraq" + 0.000*"war" + 0.000*"new" + 0.000*"england" + 0.000*"cup" + 0.000*"report" + 0.000*"review" + 0.000*"united" + 0.000*"blair"'),
 (1,
  '0.000*"football" + 0.000*"cd" + 0.000*"united" + 0.000*"rugby" + 0.000*"cricket" + 0.000*"new" + 0.000*"horse" + 0.000*"england" + 0.000*"cup" + 0.000*"racing"'),
 (2,
  '0.000*"review" + 0.000*"new" + 0.000*"win" + 0.000*"football" + 0.000*"letters" + 0.000*"england" + 0.000*"league" + 0.000*"cricket" + 0.000*"power" + 0.000*"london"'),
 (3,
  '0.000*"review" + 0.000*"league" + 0.000*"apr" + 0.000*"england" + 0.000*"football" + 0.000*"letters" + 0.000*"new" + 0.000*"rugby" + 0.000*"weekly" + 0.000*"face"'),
 (4,
  '0.000*"england" + 0.000*"premiership" + 0.000*"cup" + 0.000*"review" + 0.000*"new" + 0.000*"united" + 0.000*"football" + 0.000*"fa" + 0.000*"best" + 0.000*"league"'),
 (5,
  '0.000*"apr" + 0.000*"review" + 0.000*"film review" + 0.000*"film" + 0.000*"united" + 0.000*"england" + 0.000*"football" + 0.000*

## Make Topic Vectors

# Train Vectors

In [60]:
range(len(df_LDA_train))


range(0, 3623)

In [61]:
len(corpus)

3623

In [62]:
top_vecs_train = []
for i in range(len(df_LDA_train)):
    doc_topics = lda.get_document_topics(corpus[i], minimum_probability=0.0)
    doc_top_vec = [doc_topics[num][1] for num in range(20)]
    top_vecs_train.append(doc_top_vec)

print(len(top_vecs_train))
top_vecs_train[5]

3623


[0.014720522,
 0.014720522,
 0.014720522,
 0.014720522,
 0.014720522,
 0.014720522,
 0.014720522,
 0.014720522,
 0.014720522,
 0.7203101,
 0.014720522,
 0.014720522,
 0.014720522,
 0.014720522,
 0.014720522,
 0.014720522,
 0.014720522,
 0.014720522,
 0.014720522,
 0.014720522]

## TEST Vectors

In [63]:
# use the topic model from train data to get get data topic vectors
doc_word_test = tfv.fit_transform(df_LDA_test).transpose()
corpus_test = matutils.Sparse2Corpus(doc_word_test)

top_vecs_test = []
for i in range(len(df_LDA_test)):
    doc_topics_test = lda.get_document_topics(corpus_test[i], minimum_probability=0.0)
    doc_top_vec_test = [doc_topics_test[num][1] for num in range(20)]
    top_vecs_test.append(doc_top_vec_test)

print(len(top_vecs_test))
top_vecs_test[1]

478


[0.049999993,
 0.049999993,
 0.049999993,
 0.049999993,
 0.049999993,
 0.049999993,
 0.049999993,
 0.049999993,
 0.049999993,
 0.05000013,
 0.049999993,
 0.049999993,
 0.049999993,
 0.049999993,
 0.049999993,
 0.049999993,
 0.049999993,
 0.049999993,
 0.049999993,
 0.049999993]

In [64]:
# combine the two lists then convert to a Seires adding to the full dataframe for train, valid, test split.
top_vecs = pd.Series(top_vecs_train + top_vecs_test, name = 'top_vecs')

In [65]:
df['top_vecs'] = top_vecs
df.head()

,Label,news,doc_vec,news_str,top_vecs
0,0,"[operation, extract, leaked, report, scorecard...","[-0.13853689, -0.01842522, -0.011283746, 0.025...",operation extract leaked report scorecard hugh...,"[0.016551204, 0.016551204, 0.016551204, 0.0165..."
1,0,"[scorecard, best, lake, scene, leader, german,...","[-0.13274541, -0.32202816, 0.17546004, 0.15300...",scorecard best lake scene leader german sleaze...,"[0.018115353, 0.018115353, 0.018115353, 0.0181..."
2,0,"[coventry, caught, counter, flo, united, rival...","[-0.41247106, 0.4019597, 0.16694483, 0.0331079...",coventry caught counter flo united rival road ...,"[0.015861943, 0.015861943, 0.015861943, 0.0158..."
3,1,"[pilgrim, know, progress, thatcher, facing, ba...","[-0.232915, -0.15494952, 0.13468608, -0.152680...",pilgrim know progress thatcher facing ban mcil...,"[0.014361328, 0.014361328, 0.014361328, 0.0143..."
4,1,"[hitches, horlocks, beckham, united, survive, ...","[-0.33469847, -0.034216043, 0.117651, -0.06944...",hitches horlocks beckham united survive breast...,"[0.018412992, 0.018412992, 0.018412992, 0.0184..."


In [74]:
df['vectors'] = pd.Series([
    (list(df.doc_vec[row]) if not isinstance(df.doc_vec[row], float) else [df.doc_vec[row]]) +
    (list(df.top_vecs[row]) if not isinstance(df.top_vecs[row], float) else [df.top_vecs[row]])
    for row in range(len(df))
])

print(len(df.vectors[0]))
print(len(df.vectors))
df.head()

120
4101


,Label,news,doc_vec,news_str,top_vecs,vectors
0,0,"[operation, extract, leaked, report, scorecard...","[-0.13853689, -0.01842522, -0.011283746, 0.025...",operation extract leaked report scorecard hugh...,"[0.016551204, 0.016551204, 0.016551204, 0.0165...","[-0.13853689, -0.01842522, -0.011283746, 0.025..."
1,0,"[scorecard, best, lake, scene, leader, german,...","[-0.13274541, -0.32202816, 0.17546004, 0.15300...",scorecard best lake scene leader german sleaze...,"[0.018115353, 0.018115353, 0.018115353, 0.0181...","[-0.13274541, -0.32202816, 0.17546004, 0.15300..."
2,0,"[coventry, caught, counter, flo, united, rival...","[-0.41247106, 0.4019597, 0.16694483, 0.0331079...",coventry caught counter flo united rival road ...,"[0.015861943, 0.015861943, 0.015861943, 0.0158...","[-0.41247106, 0.4019597, 0.16694483, 0.0331079..."
3,1,"[pilgrim, know, progress, thatcher, facing, ba...","[-0.232915, -0.15494952, 0.13468608, -0.152680...",pilgrim know progress thatcher facing ban mcil...,"[0.014361328, 0.014361328, 0.014361328, 0.0143...","[-0.232915, -0.15494952, 0.13468608, -0.152680..."
4,1,"[hitches, horlocks, beckham, united, survive, ...","[-0.33469847, -0.034216043, 0.117651, -0.06944...",hitches horlocks beckham united survive breast...,"[0.018412992, 0.018412992, 0.018412992, 0.0184...","[-0.33469847, -0.034216043, 0.117651, -0.06944..."


## Modeling


In [75]:
from sklearn import  svm, naive_bayes, neighbors, ensemble
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from sklearn.metrics import f1_score, classification_report, accuracy_score

lr_model = LogisticRegression()
nb_model = naive_bayes.GaussianNB()
knn_model = neighbors.KNeighborsClassifier()
svc_model = svm.SVC(probability=True, gamma="scale")
rf_model = ensemble.RandomForestClassifier(n_estimators=100)
et_model = ensemble.ExtraTreesClassifier(n_estimators=100)
ada_model = ensemble.AdaBoostClassifier()
xgb_model = xgb.XGBClassifier(max_depth=50, n_estimators=80, learning_rate=0.1, colsample_bytree=.7, gamma=0, 
                              reg_alpha=4, objective='binary:logistic', eta=0.3, silent=1, subsample=0.8)

models = ["lr_model", "nb_model", "knn_model", "svc_model", "rf_model", "et_model", "ada_model", "xgb_model"]

In [76]:
def baseline_model_filter(modellist, X, y):
    ''' 1. split the train data further into train and validation (17%). 
        2. fit the train data into each model of the model list
        3. get the classification report based on the model performance on validation data
    '''
    X_train, X_valid, y_train, y_valid = train_test_split(X,y, test_size = 0.17, random_state = 100)
    for model_name in modellist:
        curr_model = eval(model_name)
        curr_model.fit(X_train, y_train) 
        print(f'{model_name} \n report:{classification_report(y_valid, curr_model.predict(X_valid))}')

In [78]:
df.to_csv('modified4.csv',index=False)

## doc2vec + topic vectors

In [79]:
X = np.array(list(df.vectors[:1688]))
y = np.array(list(df.Label[:1688]))

baseline_model_filter(models, X, y)

C:\Users\harsh\anaconda3\envs\mygpu\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


lr_model 
 report:              precision    recall  f1-score   support

           0       0.54      0.45      0.49       142
           1       0.54      0.62      0.58       145

    accuracy                           0.54       287
   macro avg       0.54      0.54      0.53       287
weighted avg       0.54      0.54      0.53       287

nb_model 
 report:              precision    recall  f1-score   support

           0       0.48      0.68      0.56       142
           1       0.48      0.29      0.36       145

    accuracy                           0.48       287
   macro avg       0.48      0.48      0.46       287
weighted avg       0.48      0.48      0.46       287

knn_model 
 report:              precision    recall  f1-score   support

           0       0.50      0.44      0.46       142
           1       0.51      0.57      0.53       145

    accuracy                           0.50       287
   macro avg       0.50      0.50      0.50       287
weighted avg       

## topic vectors only

In [80]:
X = np.array(list(df.top_vecs[:1688]))
y = np.array(list(df.Label[:1688]))

baseline_model_filter(models, X, y)

C:\Users\harsh\anaconda3\envs\mygpu\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\harsh\anaconda3\envs\mygpu\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\harsh\anaconda3\envs\mygpu\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


lr_model 
 report:              precision    recall  f1-score   support

           0       0.00      0.00      0.00       142
           1       0.51      1.00      0.67       145

    accuracy                           0.51       287
   macro avg       0.25      0.50      0.34       287
weighted avg       0.26      0.51      0.34       287

nb_model 
 report:              precision    recall  f1-score   support

           0       0.49      0.80      0.61       142
           1       0.49      0.19      0.28       145

    accuracy                           0.49       287
   macro avg       0.49      0.49      0.44       287
weighted avg       0.49      0.49      0.44       287

knn_model 
 report:              precision    recall  f1-score   support

           0       0.45      0.43      0.44       142
           1       0.47      0.49      0.48       145

    accuracy                           0.46       287
   macro avg       0.46      0.46      0.46       287
weighted avg       

C:\Users\harsh\anaconda3\envs\mygpu\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\harsh\anaconda3\envs\mygpu\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\harsh\anaconda3\envs\mygpu\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


svc_model 
 report:              precision    recall  f1-score   support

           0       0.00      0.00      0.00       142
           1       0.51      1.00      0.67       145

    accuracy                           0.51       287
   macro avg       0.25      0.50      0.34       287
weighted avg       0.26      0.51      0.34       287

rf_model 
 report:              precision    recall  f1-score   support

           0       0.50      0.51      0.50       142
           1       0.51      0.50      0.50       145

    accuracy                           0.50       287
   macro avg       0.50      0.50      0.50       287
weighted avg       0.50      0.50      0.50       287

et_model 
 report:              precision    recall  f1-score   support

           0       0.49      0.51      0.50       142
           1       0.50      0.49      0.50       145

    accuracy                           0.50       287
   macro avg       0.50      0.50      0.50       287
weighted avg       